# 1. Environment Setup and Library Imports

In this section, we import all necessary libraries and configure the computational environment.

In [ ]:
# Core PyTorch library - the main framework for building neural networks
import torch
import torch.nn as nn  # Neural network building blocks (layers, loss functions)
import torch.optim as optim  # Optimization algorithms (SGD, Adam, etc.)

# For loading datasets and transforming images
import torchvision
import torchvision.transforms as transforms

# For visualizing our data and results
import matplotlib.pyplot as plt

# For numerical operations and statistical analysis
import numpy as np

# For counting class distributions
from collections import Counter

print("✅ All libraries imported successfully!")

In [ ]:
# Device configuration
# Check if GPU is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Set random seeds for reproducibility
# This ensures that results are consistent across different runs
torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch version: {torch.__version__}")
print("=" * 50)
print("Environment configured successfully!")
print("=" * 50)

# 2. Data Loading and Initial Exploration

We begin by loading the Fashion-MNIST dataset without any transformations (except conversion to tensor) to understand the raw data characteristics.

In [ ]:
print("=" * 50)
print("LOADING FASHION-MNIST DATASET")
print("=" * 50)

# Initial transformation: only convert to tensor
# This preserves the original pixel value range [0, 1]
raw_transform = transforms.Compose([
    transforms.ToTensor()  # Converts PIL Image to tensor, scales to [0, 1]
])

# Load training dataset
train_dataset_raw = torchvision.datasets.FashionMNIST(
    root='../data',  # Directory to store/load the dataset
    train=True,      # Load training data
    download=True,   # Download if not present
    transform=raw_transform
)

# Load test dataset
test_dataset_raw = torchvision.datasets.FashionMNIST(
    root='../data',
    train=False,     # Load test data
    download=True,
    transform=raw_transform
)

print("✅ Dataset downloaded and loaded successfully!")
print(f"Training samples: {len(train_dataset_raw):,}")
print(f"Test samples: {len(test_dataset_raw):,}")

# 3. Exploratory Data Analysis (EDA)

Comprehensive analysis of the dataset to understand its characteristics, distribution, and properties.

## 3.1 Basic Dataset Information

In [ ]:
print("\n" + "=" * 50)
print("EXPLORATORY DATA ANALYSIS (EDA)")
print("=" * 50)

# Class names mapping
# Each label (0-9) corresponds to a specific clothing category
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print("\n📊 DATASET OVERVIEW:")
print("-" * 50)
print(f"Training samples: {len(train_dataset_raw):,}")
print(f"Test samples: {len(test_dataset_raw):,}")
print(f"Total samples: {len(train_dataset_raw) + len(test_dataset_raw):,}")

# Examine a single sample to understand data structure
sample_image, sample_label = train_dataset_raw[0]
print(f"\n📐 IMAGE PROPERTIES:")
print(f"Shape: {sample_image.shape}")  # Expected: [1, 28, 28]
print(f"  - Channels: {sample_image.shape[0]} (grayscale)")
print(f"  - Height: {sample_image.shape[1]} pixels")
print(f"  - Width: {sample_image.shape[2]} pixels")
print(f"  - Total pixels per image: {28 * 28} = 784")
print(f"Data type: {sample_image.dtype}")
print(f"\n🏷️  LABEL PROPERTIES:")
print(f"Label type: {type(sample_label)}")
print(f"Label value: {sample_label}")
print(f"Corresponding class: {classes[sample_label]}")
print(f"\n📊 Number of classes: {len(classes)}")

## 3.2 Visual Inspection of Classes

Visualizing samples from each class helps verify that:
- Images are clear and recognizable
- Labels are correctly assigned
- Image quality is sufficient for classification

In [ ]:
print("\n🖼️  VISUALIZING SAMPLE IMAGES FROM EACH CLASS")

# Create a 2x5 grid to display one sample from each of the 10 classes
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Representative Samples from Each Class', fontsize=16, fontweight='bold')

for idx, class_name in enumerate(classes):
    # Find the first occurrence of each class
    for i in range(len(train_dataset_raw)):
        img, label = train_dataset_raw[i]
        if label == idx:
            # Calculate subplot position
            row = idx // 5
            col = idx % 5
            
            # Display image
            axes[row, col].imshow(img.squeeze(), cmap='gray')
            axes[row, col].set_title(f'{idx}: {class_name}', fontsize=10)
            axes[row, col].axis('off')
            break

plt.tight_layout()
plt.show()

print("✅ Visual inspection complete")
print("   - All images are grayscale with black background")
print("   - Labels correspond correctly to clothing types")
print("   - Image quality is sufficient for classification")

## 3.3 Class Distribution Analysis

Analyzing class balance is crucial because:
- **Imbalanced datasets** can lead to biased models
- Models may favor the majority class
- Fashion-MNIST is designed to be balanced (6000 samples per class in training)
- Verification ensures data integrity

In [ ]:
print("\n📈 CLASS DISTRIBUTION ANALYSIS")
print("=" * 50)

# Extract all labels from datasets
train_labels = [label for _, label in train_dataset_raw]
test_labels = [label for _, label in test_dataset_raw]

# Count occurrences of each class
train_counts = Counter(train_labels)
test_counts = Counter(test_labels)

# Display training set distribution
print("\n📚 TRAINING SET DISTRIBUTION:")
print("-" * 70)
print(f"{'Class':<8} {'Name':<15} {'Count':<10} {'Percentage':<12}")
print("-" * 70)
for class_idx in range(10):
    count = train_counts[class_idx]
    percentage = (count / len(train_dataset_raw)) * 100
    print(f"{class_idx:<8} {classes[class_idx]:<15} {count:<10} {percentage:.2f}%")

print(f"\n{'TOTAL':<8} {'':<15} {len(train_dataset_raw):<10} 100.00%")

# Display test set distribution
print("\n🧪 TEST SET DISTRIBUTION:")
print("-" * 70)
print(f"{'Class':<8} {'Name':<15} {'Count':<10} {'Percentage':<12}")
print("-" * 70)
for class_idx in range(10):
    count = test_counts[class_idx]
    percentage = (count / len(test_dataset_raw)) * 100
    print(f"{class_idx:<8} {classes[class_idx]:<15} {count:<10} {percentage:.2f}%")

print(f"\n{'TOTAL':<8} {'':<15} {len(test_dataset_raw):<10} 100.00%")

# Visual representation of distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Training set bar chart
train_counts_list = [train_counts[i] for i in range(10)]
ax1.bar(range(10), train_counts_list, color='steelblue', edgecolor='black', alpha=0.7)
ax1.set_xlabel('Class Index', fontsize=11)
ax1.set_ylabel('Number of Images', fontsize=11)
ax1.set_title('Training Set - Class Distribution', fontsize=13, fontweight='bold')
ax1.set_xticks(range(10))
ax1.set_xticklabels([f'{i}\n{classes[i][:8]}' for i in range(10)], rotation=45, ha='right')
ax1.grid(axis='y', alpha=0.3)
ax1.axhline(y=6000, color='red', linestyle='--', linewidth=1, label='Expected (6000)')
ax1.legend()

# Test set bar chart
test_counts_list = [test_counts[i] for i in range(10)]
ax2.bar(range(10), test_counts_list, color='coral', edgecolor='black', alpha=0.7)
ax2.set_xlabel('Class Index', fontsize=11)
ax2.set_ylabel('Number of Images', fontsize=11)
ax2.set_title('Test Set - Class Distribution', fontsize=13, fontweight='bold')
ax2.set_xticks(range(10))
ax2.set_xticklabels([f'{i}\n{classes[i][:8]}' for i in range(10)], rotation=45, ha='right')
ax2.grid(axis='y', alpha=0.3)
ax2.axhline(y=1000, color='red', linestyle='--', linewidth=1, label='Expected (1000)')
ax2.legend()

plt.tight_layout()
plt.show()

print("\n✅ CONCLUSION:")
print("   - Dataset is perfectly balanced")
print("   - Each class has exactly 6000 training samples")
print("   - Each class has exactly 1000 test samples")
print("   - No bias concern from class imbalance")

## 3.4 Image Consistency Verification

Neural networks require consistent input dimensions. We verify that all images have the same shape.

In [ ]:
print("\n✅ IMAGE CONSISTENCY CHECK")
print("=" * 50)

# Expected shape for Fashion-MNIST
expected_shape = (1, 28, 28)

# Check a sample of images (checking all 60,000 would be redundant)
sample_size = 100
all_consistent = True

for i in range(sample_size):
    img, _ = train_dataset_raw[i]
    if img.shape != expected_shape:
        print(f"❌ Image {i} has unexpected shape: {img.shape}")
        all_consistent = False
        break

if all_consistent:
    print(f"✅ All images have consistent shape: {expected_shape}")
    print(f"\n📐 Image dimensions:")
    print(f"   - Channels: {expected_shape[0]} (grayscale)")
    print(f"   - Height: {expected_shape[1]} pixels")
    print(f"   - Width: {expected_shape[2]} pixels")
    print(f"   - Total pixels per image: {28 * 28} = 784")
    print(f"\n   This is crucial for neural network input compatibility")

## 3.5 Pixel Value Statistical Analysis

Understanding the distribution of pixel values is essential for:
1. **Normalization strategy:** Determines how to scale inputs
2. **Model convergence:** Properly scaled inputs train faster
3. **Activation function choice:** Input range affects layer outputs

We analyze:
- Value range (min, max)
- Central tendency (mean, median)
- Spread (standard deviation)
- Distribution shape (histogram)

In [ ]:
print("\n🔢 PIXEL VALUE STATISTICAL ANALYSIS")
print("=" * 50)

# Sample random images to analyze pixel distribution
sample_size = 1000
sample_indices = np.random.choice(len(train_dataset_raw), sample_size, replace=False)

# Collect all pixel values from sampled images
pixel_values = []
for idx in sample_indices:
    img, _ = train_dataset_raw[idx]
    # Flatten the image (28x28 -> 784) and convert to numpy array
    pixel_values.extend(img.flatten().numpy())

pixel_values = np.array(pixel_values)

# Calculate statistics
min_val = pixel_values.min()
max_val = pixel_values.max()
mean_val = pixel_values.mean()
std_val = pixel_values.std()
median_val = np.median(pixel_values)

print(f"\n📊 Statistics from {sample_size} random images:")
print(f"   Total pixels analyzed: {len(pixel_values):,}")
print(f"   (Expected: {sample_size} × 784 = {sample_size * 784:,})")
print(f"\n📈 Descriptive Statistics:")
print(f"   Min value:           {min_val:.4f}")
print(f"   Max value:           {max_val:.4f}")
print(f"   Mean (μ):            {mean_val:.4f}")
print(f"   Std Dev (σ):         {std_val:.4f}")
print(f"   Median:              {median_val:.4f}")

# Visualize pixel value distribution
plt.figure(figsize=(12, 5))

# Histogram
plt.subplot(1, 2, 1)
plt.hist(pixel_values, bins=50, color='purple', alpha=0.7, edgecolor='black')
plt.xlabel('Pixel Value', fontsize=11)
plt.ylabel('Frequency', fontsize=11)
plt.title('Distribution of Pixel Values\n(Sample of 1000 Images)', fontsize=13, fontweight='bold')
plt.axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean = {mean_val:.3f}')
plt.axvline(median_val, color='green', linestyle='--', linewidth=2, label=f'Median = {median_val:.3f}')
plt.grid(axis='y', alpha=0.3)
plt.legend()

# Box plot for additional insight
plt.subplot(1, 2, 2)
plt.boxplot(pixel_values, vert=True)
plt.ylabel('Pixel Value', fontsize=11)
plt.title('Box Plot of Pixel Values', fontsize=13, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 KEY OBSERVATIONS:")
print(f"   1. Pixel values are in the range [0, 1] (after ToTensor transformation)")
print(f"   2. Distribution is heavily skewed toward 0 (black background)")
print(f"   3. Mean = {mean_val:.4f} indicates most pixels are dark")
print(f"   4. Low median = {median_val:.4f} confirms background dominance")
print(f"   5. Actual clothing information is in higher pixel values (0.5-1.0)")

## 3.6 EDA Summary

Summary of key findings from exploratory data analysis.

In [ ]:
print("\n" + "=" * 60)
print("EDA SUMMARY - KEY FINDINGS")
print("=" * 60)

print("\n📊 DATASET CHARACTERISTICS:")
print(f"   • Training samples:    {len(train_dataset_raw):,}")
print(f"   • Test samples:        {len(test_dataset_raw):,}")
print(f"   • Total samples:       {len(train_dataset_raw) + len(test_dataset_raw):,}")
print(f"   • Number of classes:   {len(classes)}")

print("\n📐 IMAGE SPECIFICATIONS:")
print(f"   • Shape:               {expected_shape} (C × H × W)")
print(f"   • Color space:         Grayscale (1 channel)")
print(f"   • Dimensions:          28 × 28 pixels")
print(f"   • Pixels per image:    784")

print("\n📈 DATA DISTRIBUTION:")
print(f"   • Class balance:       Perfect (6000 per class in training)")
print(f"   • Pixel value range:   [0, 1]")
print(f"   • Mean pixel value:    {mean_val:.4f}")
print(f"   • Std deviation:       {std_val:.4f}")

print("\n✅ DATA QUALITY ASSESSMENT:")
print("   ✓ No missing values")
print("   ✓ Consistent image dimensions")
print("   ✓ Balanced class distribution")
print("   ✓ Sufficient image quality for classification")
print("   ✓ Ready for preprocessing and model training")

print("\n" + "=" * 60)

# 4. Normalization Strategy and Decision Making

## 4.1 Theoretical Background

**Why Normalize?**

Normalization transforms data to have specific statistical properties, typically:
- Mean (μ) ≈ 0
- Standard deviation (σ) ≈ 1

**Benefits:**
1. **Faster convergence:** Gradient descent converges more quickly
2. **Numerical stability:** Prevents vanishing/exploding gradients
3. **Fair feature comparison:** All pixels contribute equally
4. **Better initialization:** Weights initialize in appropriate range

**Normalization Formula:**

$$x_{normalized} = \frac{x - \mu}{\sigma}$$

Where:
- $x$ = original pixel value
- $\mu$ = mean to subtract (centering parameter)
- $\sigma$ = standard deviation to divide by (scaling parameter)

## 4.2 Normalization Approaches

We consider two strategies:

### Approach A: Standard Normalization
- **Parameters:** μ = 0.5, σ = 0.5
- **Rationale:** Common convention in tutorials/literature
- **Result:** Transforms [0, 1] → approximately [-1, 1]
- **Issue:** Assumes mean = 0.5, but our actual mean = 0.2913

### Approach B: Custom Normalization
- **Parameters:** μ = 0.2913, σ = 0.3552 (from EDA)
- **Rationale:** Matches actual data distribution
- **Result:** Properly centers data at 0
- **Advantage:** Statistically correct for this specific dataset

## 4.3 Mathematical Verification

For **Standard Normalization** (μ=0.5, σ=0.5):
$$x_{norm} = \frac{0.2913 - 0.5}{0.5} = -0.417$$

Our data centers at **-0.417**, not 0! ❌

For **Custom Normalization** (μ=0.2913, σ=0.3552):
$$x_{norm} = \frac{0.2913 - 0.2913}{0.3552} = 0$$

Our data centers at exactly **0**! ✅

## 4.4 Decision

We will implement **both approaches** and compare their performance to make an empirical, data-driven decision.

In [ ]:
print("=" * 60)
print("NORMALIZATION EXPERIMENT SETUP")
print("=" * 60)

print("\n🔬 EXPERIMENTAL DESIGN:")
print("\nWe will create two parallel pipelines:")
print("\n📊 Pipeline A - Standard Normalization")
print("   • Transform: Normalize(mean=0.5, std=0.5)")
print("   • Rationale: Industry standard, widely used")
print("   • Expected range: ≈ [-1, 1]")

print("\n📊 Pipeline B - Custom Normalization")
print("   • Transform: Normalize(mean=0.2913, std=0.3552)")
print("   • Rationale: Matches actual data statistics from EDA")
print("   • Expected range: Properly centered at 0")

print("\n📋 COMPARISON METRICS:")
print("   1. Training convergence speed (loss per epoch)")
print("   2. Final test accuracy")
print("   3. Training stability (loss variance)")

print("\n💡 HYPOTHESIS:")
print("   Custom normalization should perform slightly better due to")
print("   proper centering of our specific data distribution.")

# 5. Data Preprocessing - Implementation

Creating two separate preprocessing pipelines for experimental comparison.

In [ ]:
print("\n" + "=" * 60)
print("IMPLEMENTING PREPROCESSING PIPELINES")
print("=" * 60)

# Pipeline A: Standard Normalization
transform_standard = transforms.Compose([
    transforms.ToTensor(),                    # Convert to tensor: [0, 255] → [0, 1]
    transforms.Normalize((0.5,), (0.5,))     # Normalize: [0, 1] → ≈[-1, 1]
])

# Pipeline B: Custom Normalization (using EDA statistics)
transform_custom = transforms.Compose([
    transforms.ToTensor(),                           # Convert to tensor: [0, 255] → [0, 1]
    transforms.Normalize((0.2913,), (0.3552,))      # Normalize using actual μ and σ
])

print("\n✅ Two transformation pipelines created successfully!")

print("\n📝 PIPELINE A - Standard Transform:")
print("   Step 1: ToTensor() → converts to [0, 1] range")
print("   Step 2: Normalize(mean=0.5, std=0.5)")
print("   Formula: (pixel - 0.5) / 0.5")

print("\n📝 PIPELINE B - Custom Transform:")
print("   Step 1: ToTensor() → converts to [0, 1] range")
print("   Step 2: Normalize(mean=0.2913, std=0.3552)")
print("   Formula: (pixel - 0.2913) / 0.3552")
print("   This centers our data at true mean = 0")

In [ ]:
print("\n" + "=" * 60)
print("LOADING DATASETS WITH TRANSFORMATIONS")
print("=" * 60)

# Dataset A: Standard Normalization
print("\n📦 Loading Dataset A (Standard Normalization)...")
train_dataset_A = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform_standard
)

test_dataset_A = torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform_standard
)
print("✅ Dataset A loaded successfully")

# Dataset B: Custom Normalization
print("\n📦 Loading Dataset B (Custom Normalization)...")
train_dataset_B = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform_custom
)

test_dataset_B = torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform_custom
)
print("✅ Dataset B loaded successfully")

print("\n" + "=" * 60)
print("Both datasets ready for DataLoader creation")
print("=" * 60)

In [ ]:
print("\n" + "=" * 60)
print("CREATING DATALOADERS")
print("=" * 60)

# Batch size configuration
# Batch size determines how many samples are processed before updating weights
batch_size = 64

# DataLoaders for Dataset A (Standard Normalization)
train_loader_A = torch.utils.data.DataLoader(
    dataset=train_dataset_A,
    batch_size=batch_size,
    shuffle=True,        # Shuffle training data for better generalization
    num_workers=0,       # Number of subprocesses for data loading
    pin_memory=False     # Pin memory for faster GPU transfer (if using GPU)
)

test_loader_A = torch.utils.data.DataLoader(
    dataset=test_dataset_A,
    batch_size=batch_size,
    shuffle=False,       # Don't shuffle test data (order doesn't matter)
    num_workers=0,
    pin_memory=False
)

# DataLoaders for Dataset B (Custom Normalization)
train_loader_B = torch.utils.data.DataLoader(
    dataset=train_dataset_B,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=False
)

test_loader_B = torch.utils.data.DataLoader(
    dataset=test_dataset_B,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=False
)

print("\n✅ ALL DATALOADERS CREATED SUCCESSFULLY")
print("=" * 60)
print(f"\n📊 DataLoader Configuration:")
print(f"   • Batch size:                 {batch_size}")
print(f"   • Training batches (per set): {len(train_loader_A)}")
print(f"   • Test batches (per set):     {len(test_loader_A)}")
print(f"   • Samples per epoch:          {len(train_dataset_A):,}")

# Calculate iteration details
total_training_iterations = len(train_loader_A)
samples_per_iteration = batch_size
print(f"\n🔄 Per Epoch:")
print(f"   • Iterations: {total_training_iterations}")
print(f"   • Samples per iteration: {samples_per_iteration}")
print(f"   • Last batch may have fewer samples: {len(train_dataset_A) % batch_size} samples")

print("\n🔬 Ready to train both models for comparison!")

## 5.1 Visual Verification of Normalization

Comparing the same image under both normalization strategies to verify transformations are applied correctly.

In [ ]:
print("\n" + "=" * 60)
print("VISUAL COMPARISON OF NORMALIZATION STRATEGIES")
print("=" * 60)

# Select the same image from both datasets (using index 42 for consistency)
img_A, label_A = train_dataset_A[42]
img_B, label_B = train_dataset_B[42]
img_raw, label_raw = train_dataset_raw[42]

# Denormalization functions (reverse the normalization for visualization)
def denormalize_standard(img):
    """Reverse standard normalization: x_original = x_norm * std + mean"""
    return img * 0.5 + 0.5

def denormalize_custom(img):
    """Reverse custom normalization"""
    return img * 0.3552 + 0.2913

# Create visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle(f'Normalization Comparison - Same Image (Label: {classes[label_raw]})', 
             fontsize=16, fontweight='bold')

# Original image (raw, only ToTensor applied)
axes[0].imshow(img_raw.squeeze(), cmap='gray')
axes[0].set_title('Original\n[0, 1] range\nNo normalization', fontsize=11)
axes[0].axis('off')
axes[0].text(0.5, -0.15, f'Mean: {img_raw.mean().item():.3f}', 
             ha='center', transform=axes[0].transAxes, fontsize=10)

# Standard normalized
img_A_display = denormalize_standard(img_A)
axes[1].imshow(img_A_display.squeeze(), cmap='gray')
axes[1].set_title('Standard Normalization\nN(0.5, 0.5)\n≈[-1, 1] range', fontsize=11)
axes[1].axis('off')
axes[1].text(0.5, -0.15, f'Normalized mean: {img_A.mean().item():.3f}', 
             ha='center', transform=axes[1].transAxes, fontsize=10)

# Custom normalized
img_B_display = denormalize_custom(img_B)
axes[2].imshow(img_B_display.squeeze(), cmap='gray')
axes[2].set_title('Custom Normalization\nN(0.291, 0.355)\nCentered at 0', fontsize=11)
axes[2].axis('off')
axes[2].text(0.5, -0.15, f'Normalized mean: {img_B.mean().item():.3f}', 
             ha='center', transform=axes[2].transAxes, fontsize=10)

plt.tight_layout()
plt.show()

print("\n💡 KEY OBSERVATIONS:")
print(f"   • All three images appear visually identical (after denormalization)")
print(f"   • Original mean: {img_raw.mean().item():.4f}")
print(f"   • Standard normalized mean: {img_A.mean().item():.4f} (not centered at 0)")
print(f"   • Custom normalized mean: {img_B.mean().item():.4f} (closer to 0)")
print(f"\n   The different scaling affects how the neural network processes them!")

## 5.2 Dimension Verification

Final verification that transformations maintain correct tensor dimensions for neural network input.

In [ ]:
print("\n" + "=" * 60)
print("VERIFYING TENSOR DIMENSIONS")
print("=" * 60)

# Test with Dataset A
sample_img, sample_label = train_dataset_A[0]

print("\n📐 Tensor Properties:")
print(f"   • Shape: {sample_img.shape}")
print(f"   • Expected: torch.Size([1, 28, 28])")
print(f"   • Match: {'✅ Yes' if sample_img.shape == torch.Size([1, 28, 28]) else '❌ No'}")

print(f"\n📊 Value Range:")
print(f"   • Min: {sample_img.min().item():.4f}")
print(f"   • Max: {sample_img.max().item():.4f}")
print(f"   • Mean: {sample_img.mean().item():.4f}")

print(f"\n🏷️  Label:")
print(f"   • Value: {sample_label}")
print(f"   • Type: {type(sample_label)}")
print(f"   • Class: {classes[sample_label]}")

print("\n✅ All dimensions verified - ready for model training!")
print("=" * 60)

# Summary of Current Progress

## Completed Steps:

1. ✅ **Environment Setup**
   - Imported all necessary libraries
   - Configured device (CPU/GPU)
   - Set random seeds for reproducibility

2. ✅ **Data Loading**
   - Downloaded Fashion-MNIST dataset
   - Verified dataset integrity

3. ✅ **Exploratory Data Analysis (EDA)**
   - Analyzed dataset structure and dimensions
   - Visualized sample images from each class
   - Verified class distribution (balanced)
   - Checked image consistency
   - Performed statistical analysis of pixel values

4. ✅ **Normalization Strategy**
   - Analyzed theoretical background
   - Designed two experimental approaches
   - Created transformation pipelines

5. ✅ **Data Preprocessing**
   - Implemented standard normalization pipeline (Dataset A)
   - Implemented custom normalization pipeline (Dataset B)
   - Created DataLoaders for both approaches
   - Verified transformations visually and numerically

## Next Steps:

6. **Model Architecture Design**
   - Define neural network structure
   - Choose layers, activation functions, dropout
   
7. **Training Implementation**
   - Define loss function and optimizer
   - Implement training loop
   - Train both models (A and B)
   
8. **Evaluation and Comparison**
   - Evaluate model performance
   - Compare normalization strategies
   - Analyze results

---

**Current Status:** Ready to proceed with model architecture design and training.